# PDE Modelling 

## Objectives 
* Make a model that describes cell growth and signalling at the scale of colonies 

## Model considerations 
* Species
    1. Cell density 
    1. Nutrient density 
    1. Environmental AHL (considered equal to intracellular concentration)
    1. Synthase / GFP
    1. Repressor or degradase
* Reactions 
    1. cell growth and diffusion 
        * Cells diffuse very slowly
        * nutrient-dependent growth (from Liu et al 2011, Science) 
        $$  $$ 
    1. Transcriptional activation
        * Basal protein expression -> scaled by repression? probably
        * Activation by internal AHL 
        * Use Hill function $$H(A(t), n, k) = \frac{A(t)^2}{k^n + A(t)^n}$$
        * Activation term, with basal expression and expression rate x
        $$X(A(t), n, k, b, x) = x\frac{A(t)^2}{k^n + A(t)^n} + b$$
    1. Transcriptional repression
        * Assume activation is AND-like, meaning that repression trumps activation
        * Use 'repression' Hill function $$H_n(R(t), n, k) = \frac{k^n}{k^n + R(t)^n}$$
        * Rather than considering protein concentrations within cells, calculate protein concentrations as produced by the bulk of cells. Expression is therefore proportional to cell density.
    1. Dilution and degradation 
        * Assume that GFP/Synthase proteins are degradation tagged
        * Degradase is not tagged, so does not have a degradation term
    1. Diffusion 
        * Here, you're going to use convoultion of the diffusion kernel
        * Diffusion in/out of cell is considered faster than spatial diffusion at these scales
    1. Parameters
        * We are also assuming, for the moment, that each time point is 6 minutes. Parameters with time dimensions shown below may use different units than the parameter from the cited paper.
        * dx: Length modification of diffusion terms. In the compartmental model, diffusion is calculated via Ficks' first law, where the flux between two adjacent compartments is equal to the flux multiplied by the area of the interface between the components :  
        $\frac{\mathrm{d} C}{\mathrm{d} t} $ 
        in continuous form gives up 
        $\Delta C = D \frac{A}{V} \frac{\Delta C}{\Delta x} = D \frac{2.25 \cdot 5 \cdot \mathrm{scale}^2 \mathrm{mm}^2}{\mathrm{scale} \cdot 2.25^2 \cdot 5 \mathrm{mm}^3} \frac{\Delta C \cdot \mathrm{scale}}{2.25 \mathrm{mm}} = \frac{D \Delta C \mathrm{scale}^2}{2.25^2 \mathrm{mm}^2}$. the dx parameter below is the symbol $A$ in this equation.
        * Dc : Diffusion rate for cells. $7\frac{mm^2}{min}$
        * rc : Division rate of cells. $\frac{1.14}{min}$
        * Kn : Half-point of nutrient availability. 75
        * Dn : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * kn : Consumption rate of nutrient by cells
        * Da : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * xa : Synthesis rate of AHL. 
        * xs : Expression rate of protein. 
        * ha : Hill coefficient of AHL-inducible expression.
        * ka : Half-point of AHL-inducible expression. 
        * pa : Degradation rate of AHL.
        * leak : Leaky expression rate of protein. 
        


In [1]:
# imports
from __future__ import division, print_function
import numpy as np
import pandas as pd
import os
import sys
import string
import scipy.integrate as itg
import scipy.optimize as opt
import scipy.ndimage as ndi
import matplotlib.pyplot as plt 
import matplotlib.animation as anm
import skimage.measure
import numba
import gc
from multiprocessing import Pool, Process

%load_ext line_profiler

from IPython.display import HTML

%matplotlib inline

## 2D Discrete Laplacian

In continuous form : 
$$ U_t = \triangle U - \lambda U $$

In discrete form, for point $i$ : 
$$ \Delta U_i = \sum_{1 = w(i,j)}\omega(i,j)(U_i - U_j) - \lambda U_i $$

Use discrete laplacian approximation w/o diagonals for grid spacing, so that we can have zero-flux  boundary conditions. 

$$ L = 
 \begin{pmatrix}
  0 & 1 & 0 \\
  1 & -4 & 1 \\
  0 & 1 & 0 
 \end{pmatrix} $$

I use a convolution function to calculate the diffusion terms. 

# Helper functions used to define the arenas 
### Needs
* read excel or csv files 
* rescaling arrays and contents 
* convert row/col to array index


* disk function, projects circular areas onto an input grid 
* 

In [2]:
def disk(A, center, radius):
    h, w = A.shape
    ind_mat = np.zeros((h, w, 2))
    cx, cy = center
    for i in range(h):
        ind_mat[i,:,0] = np.power(np.arange(w) - cx, 2)
    
    for i in range(w):
        ind_mat[:,i,1] = np.power(np.arange(h) - cy, 2)
    
    outmat = (ind_mat[:,:,0] + ind_mat[:,:,1]) < radius**2
    return ndi.filters.gaussian_filter(outmat, radius/3)


let_dict = dict(zip(string.ascii_uppercase, np.arange(0,26)))

scale = 5
scale_s = np.int(scale/2)
n_w = 48 * scale
n_h = 32 * scale

species = 7 # rc_cells, cr_cells, nutrients, AHL_c, AHL_r, synthase_c, synthase_r
tup = np.array([species, n_h, n_w])
rc_i, cr_i, n_i, rhl_i, cin_i, rhli_i, cini_i = np.arange(species)

# Make empty array
A = np.zeros((species, n_h, n_w), dtype=np.float32,order='C') + 1e-7

# Set initial conditions
# Nutrients. All at 100
A[n_i,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)

tmax = 10
t_points = np.int(tmax*10)

# units : L = mm, T = minutes, concentration in nM = moles / mm^3
# Da = 6 - 1.2 E-2
#LEGACY
# Params :    dx,                          Dc,    rc,  Kn,   Dn,   kn,  Da,  xa,  xs,  ha,  ka, 
p0 = np.array([np.power((scale/2.25),2),   5e-3, 6e-3,  75,  8e-3,  2, 8e-2, 1e3, 2e-0, 2.3, 40,    
          # pa,   leak   od0
             5e-5, 1e-8, 0.5], dtype=np.float32)

# Params :    dx,                          Dc,    rc,  Kn,   Dn,    kn,  Da,  xa,  xs,  ha,  ka, 
p0 = np.array([np.power((scale/2.25),2),   1e-4, 6e-3,  75,  8e-3,  4, 2e-2, 1e3, 2e-0, 2.3, 40,
# hC, kC, pa,   leak   od0
 2.3, 40, 5e-5, 1e-8, 0.5], dtype=np.float32)

# Change parameter values above. The function definitions inherit the parameter values defined here.
dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od0 = p0

#@numba.jit('void(float32[:,:,:],float32[:,:,:])', nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def calc_diffusion(A, D):
    # Middle
    D[:,1:-1,1:-1] = A[:,1:-1, 2:] + A[:,1:-1, :-2] + A[:,:-2, 1:-1] + A[:,2:, 1:-1] - 4*A[:,1:-1, 1:-1]
    # Edges
    D[:,0,1:-1] = A[:,0, 2:] + A[:,0, :-2] + A[:,1, 1:-1] - 3*A[:,0, 1:-1]
    D[:,-1,1:-1] = A[:,-1, 2:] + A[:,-1, :-2] + A[:,-2, 1:-1] - 3*A[:,-1, 1:-1]
    D[:,1:-1,0] = A[:,2:,0] + A[:,:-2,0] + A[:,1:-1,1] - 3*A[:,1:-1,0]
    D[:,1:-1,-1] = A[:,2:,-1] + A[:,:-2,-1] + A[:,1:-1,-2] - 3*A[:,1:-1,-1]
    # Corners
    D[:,0,0] = A[:,0,1] + A[:,1,0] - 2*A[:,0,0]
    D[:,-1,0] = A[:,-1,1] + A[:,-2,0] - 2*A[:,-1,0]
    D[:,0,-1] = A[:,0,-2] + A[:,1,-1] - 2*A[:,0,-1]
    D[:,-1,-1] = A[:,-1,-2] + A[:,-2,-1] - 2*A[:,-1,-1]

#@numba.jit('float32[:,:](float32[:,:],float32,float32)',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def hill(a, n, k):
    h_ma = 1 - (1 / (1 + (a/k)**n))
    return h_ma

#@numba.jit('float32[:,:](float32[:,:],float32,float32)',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def hillN(a, n, k):
    return 1 / (1 + (a/k)**n)

#@numba.jit('void(float32[:,:,:],float32[:,:,:],float32[:,:,:],float32[:,:])',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def calc_f(y, d_y, diff_terms, nut_avail, p0):
    dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od = p0
    calc_diffusion(y, diff_terms)
    
    # Growth term
    nut_avail[:] = hill(y[n_i,:,:], 2, Kn)
    
    d_y[rc_i,:,:] = (dx)*Dc*diff_terms[rc_i,:,:] + rc * nut_avail * y[rc_i,:,:]
    d_y[cr_i,:,:] = (dx)*Dc*diff_terms[cr_i,:,:] + rc * nut_avail * y[cr_i,:,:]
    d_y[n_i,:,:] = (dx)*Dn*diff_terms[n_i,:,:] - kn * nut_avail * (y[rc_i,:,:] + y[cr_i,:,:])
    d_y[rhl_i,:,:] = (dx)*Da*diff_terms[rhl_i,:,:] + xa * y[rhli_i,:,:]*y[cr_i,:,:] - pa * y[rhl_i,:,:]
    d_y[cin_i,:,:] = (dx)*Da*diff_terms[cin_i,:,:] + xa * y[cini_i,:,:]*y[rc_i,:,:] - pa * y[cin_i,:,:]
    d_y[rhli_i,:,:] = (dx)*Dc*diff_terms[rhli_i,:,:] + xs * y[cr_i,:,:] * (hill(y[cin_i,:,:], hR, kR) + leak) * nut_avail
    d_y[cini_i,:,:] = (dx)*Dc*diff_terms[cini_i,:,:] + xs * y[rc_i,:,:] * (hill(y[rhl_i,:,:], hC, kC) + leak) * nut_avail
    

# ODE definition
#@numba.jit('float32[:](float32[:],float32[:],float32[:,:,:],float32[:,:,:],float32[:,:])', nopython=True)
@numba.jit(nopython=True)
def f(y, t, d_y, diff_terms, nut_avail, p0):
    
    y.shape = (species, n_h, n_w)
    calc_f(y, d_y, diff_terms, nut_avail, p0)
    
    return d_y.flatten()

def f_ivp(t, y, d_y, diff_terms, nut_avail, p0):
    
    y.shape = (species, n_h, n_w)
    calc_f(y, d_y, diff_terms, nut_avail, p0)
    
    return d_y.flatten()


In [3]:
def sim_omnitray(rc_spots, cr_spots, ahl_spots, p0, tmax=tmax, A=A.copy()):
    
    dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od = p0
    
    t = np.linspace(0,tmax,t_points,dtype=np.float32)

    od0 = od
    
    rc_cells = np.zeros((n_h, n_w), dtype=np.float32)
    for center in rc_spots:
        rc_cells += disk(rc_cells, scale*np.array(center), scale_s)*od0

    cr_cells = np.zeros((n_h, n_w), dtype=np.float32)
    for center in cr_spots:
        cr_cells += disk(cr_cells, scale*np.array(center), scale_s)*od0
        
    ahl_drops = np.zeros((n_h, n_w), dtype=np.float32)
    for center in ahl_spots:
        ahl_drops += disk(ahl_drops, scale*np.array(center), scale_s)*od0

    # Set initial conditions
    # rc_ells. Spotted according to the echo pick lists
    A[0,:,:] += rc_cells

    # cr_ells. Spotted according to the echo pick lists
    A[1,:,:] += cr_cells

    # Blur to make smooth colonies. This is basically cosmetic
    #A[0,:,:] = ndi.filters.gaussian_filter(A[0,:,:], scale_s)
    #A[1,:,:] = ndi.filters.gaussian_filter(A[1,:,:], scale_s)

    # Nutrients. All at 100
    #A[2,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)

    # External rhl AHL.
    #A[3,:,:] = np.zeros((n_h, n_w))

    # External cin AHL.
    A[4,:,:] += ahl_drops * 2.5e3

    # cini
    #A[5,:,:] = np.zeros((n_h, n_w))

    # rhili
    #A[6,:,:] = np.zeros((n_h, n_w))

    args=(np.zeros(A.shape, dtype=np.float32,order='C'), 
          np.zeros(A.shape, dtype=np.float32,order='C'), 
          np.zeros(A.shape[1:], dtype=np.float32,order='C'), 
          p0)
    A.shape = n_h*n_w*species
    print('Starting')
    #out = itg.odeint(f, A, t, args=args, atol=1e-6)
    f_lambda = lambda t, y : f_ivp(t, y, *args)
    out = itg.solve_ivp(f_lambda, [0, tmax], A, vectorized=True, t_eval=np.arange(tmax))
    out.shape = (t_points, species, n_h, n_w)
    print('DONE')
    return out


fn_base = "/home/jmp/data/echo_files/20170829_circuit/combo/20170829_combo_ST{}.csv"
fnames = [fn_base.format(strain) 
          for strain in [3,4]
         ]
ahl_fn = "/home/jmp/data/echo_files/20170829_circuit/AHL/20170829_combo_Cin AHL.csv"

#out = sim_omnitray(4, fnames[0], fnames[1], ahl_fn, p0)

# Take a look at one frame
out.resize((t_points,species,n_h,n_w))

print(out.shape)

plt.close('all')
fig, axs = plt.subplots(1,species, figsize=(19,5))
for i in np.arange(species):
    ax = axs[i]
    img = ax.imshow(out[-1,i,:,:], interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])
    cbar = fig.colorbar(mappable=img, ax=ax)
plt.show()

t_i = np.arange(im_t)[colony_mean[:,0].max() == colony_mean[:,0]]
masks, ls = skimage.measure.label(
    np.logical_xor(im_arr[t_i, 0, :, :] > 0.12, 
                   im_arr[t_i, 1, :, :] > 0.12), 
    connectivity=1, 
    return_num=True)
labels_vec[t_i] = labels
plt.imshow(masks[0])
print(ls)

In [4]:
# Try out FunctionAnimation approach
fn_base = "/home/jmp/data/echo_files/20170829_circuit/no AiiA/20170829_{}{}_ST{}.csv"
fnames = [
           [
              fn_base.format(letter, space, strain) 
              for space in [3]
              for strain in [4, 3]
            ]
          for letter in ['a', 'b', 'c']
         ]

ahl_fn = "/home/jmp/data/echo_files/20170829_circuit/AHL/20170829_abc3_Cin AHL.csv"
mat = pd.read_csv(ahl_fn)
well_strs = list(mat[['    Destination Well']].values[:,0])
ahl_centers = [(int(w[1:]), let_dict[w[:1]]) for w in well_strs]


dest_wells = []
for batch in fnames:
    batch_list = []
    for fn in batch:
        mat = pd.read_csv(fn)
        well_strs = list(mat[['    Destination Well']].values[:,0])
        colony_centers = [(int(w[1:]), let_dict[w[:1]]) for w in well_strs]
        batch_list.append(colony_centers)
    dest_wells.append(batch_list)

fn_inputs = []
for i in range(3):
    fn_inputs.append(dest_wells[i] + [ahl_centers, p0])

def wrapper(p):
    rc_spots, cr_spots, ahl_spots, p0 = p
    burn_in = sim_omnitray(rc_spots, cr_spots, [], p0, tmax=18*60, A=A.copy())
    overnight_t = burn_in.t
    overnight_y = burn_in.y.T
    overnight_y.shape = (len(overnight_t), species, n_h, n_w)
    out = sim_omnitray([], [], ahl_spots, p0, A=overnight_y[-1,:,:,:], tmax=4*24*60)
    exp_t = out.t
    exp_y = out.y.T
    exp_y.shape = (len(exp_t), species, n_h, n_w)
    out = np.concatenate((overnight_y, exp_y), axis=0)
    t_vec = np.concatenate((overnight_t, exp_t))
    return exp_y, exp_t
    
#with Pool(3) as p:
#    res = p.map(wrapper, fn_inputs)

a_out = wrapper(fn_inputs[0])


Starting
DONE
Starting


MemoryError: 

In [ ]:
def write_movie(out, t, fn):

    plt.close('all')

    indx = cin_i
    if len(t) < 50:
        frames = len(t)
    else:
        frames = 50
    view_masks = True

    t_points, s, h, w = out.shape
    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure()
    ax = plt.axes()
    im_arr = out[::np.int(t_points/frames),:,:,:]
    gc.collect()
    t, s, h, w = im_arr.shape
    blank_array = np.zeros([n_h, n_w])
    if view_masks:
        vmax = 7
        vmin = 0
    else:
        vmax = im_arr[:,indx,:,:].max()
        vmin = im_arr[:,indx,:,:].min()
    im = plt.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')

    # initialization function: plot the background of each frame
    def init():
        im.set_array(blank_array)
        return im,

    # animation function.  This is called sequentially
    def animate(i):
        if view_masks:
            mask, labls = skimage.measure.label(
                np.logical_xor(im_arr[i, cr_i, :, :] > 0.05, im_arr[i, rc_i, :, :] > 0.05),
                        return_num=True, connectivity=1) 
            im.set_array(mask)    
        else:
            im.set_array(im_arr[i, indx, :, :] + im_arr[i, indx-1, :, :] )
            #im.set_array(im_arr[i, indx, :, :])
        return im,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = anm.FuncAnimation(fig, animate, interval=50, frames=frames)


    # Set up formatting for the movie files
    #Writer = anm.writers['ffmpeg_file']
    #writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=900, extra_args=['-vcodec', 'libx264'])

    # save the animation as an mp4.  This requires ffmpeg or mencoder to be
    # installed.  The extra_args ensure that the x264 codec is used, so that
    # the video can be embedded in html5.  You may need to adjust this for
    # your system: for more information, see
    # http://matplotlib.sourceforge.net/api/animation_api.html
    #anim.save('animation_{}.mp4'.format(fn), extra_args=['-vcodec', 'libx264'], dpi=50, writer=writer)
    #plt.close('all')


    #anim.save('animation_{}.mp4'.format(fn), writer=writer)
    plt.close('all')
    return anim
    #HTML(anim.to_html5_video())
    
out_names = ["a", "b", "c"]
out, t = a_out
anim = write_movie(out, t, out_names[i])
HTML(anim.to_html5_video())

In [ ]:
jd = pd.read_csv('20170829_pos1_wCtime.csv')
cols = jd.columns
data_cols = cols[3:9]
m1, m2 = jd[data_cols[::2]].dropna(axis=0).T.values.max(), jd[data_cols[1::2]].dropna(axis=0).T.values.max()
plt.figure(figsize=(20,8))
clean_fit_dict = dict()
for col in data_cols[::2]:
    y, t = list(jd[[col, 'Time elapsed']].dropna(axis=0).T.values)
    y = (y-y.min())/(m1-y.min())
    plt.plot(t, y,'.', label=col)
    fit = np.polyfit(t, y, 5)
    fitfn = np.poly1d(fit)
    clean_fit_dict[col] = fitfn
    plt.plot(t, fitfn(t),'k')
    
for col in data_cols[1::2]:
    y, t = list(jd[[col, 'Time elapsed']].dropna(axis=0).T.values)
    y = (y-y.min())/(m2-y.min())
    plt.plot(t, y,'.', label=col)
    fit = np.polyfit(t, y, 5)
    fitfn = np.poly1d(fit)
    clean_fit_dict[col] = fitfn
    plt.plot(t, fitfn(t),'k')    
    
clean_fit_vec = [clean_fit_dict[key] for key in data_cols]

plt.plot([0, 70], [0.18, 0.18], '--k')
plt.legend()

In [ ]:
#traces = get_traces(out)
#print(traces.shape)
#for spec in range(species):
#    plt.plot(traces[1000::10,spec])

In [ ]:
def get_traces(out):
    t, s, h, w = out.shape
    colony_mean = np.zeros((t, s))
    for t_i in np.arange(t):
        masks, n_l = skimage.measure.label(
            np.logical_xor(out[t_i, 0, :, :] > 0.05, 
                           out[t_i, 1, :, :] > 0.05), 
            connectivity=1, 
            return_num=True)
        for l in range(0,s,2):
            colony_mean[t_i, l] = np.mean((masks==(l+1))*(out[t_i, cini_i,:,:]))
        for l in range(1,s,2):
            colony_mean[t_i, l] = np.mean((masks==(l+1))*(out[t_i, rhli_i,:,:]))
    return colony_mean

def sim_wrapper(p_in):
    xa, xs, hR, kR, hC, kC, pa, leak, od = p_in
    dx, Dc,  rc,    Kn,  Dn,   kn, Da = 2.25/np.power(scale,2),   2e-3, 6e-3,  75,  4e-1,  2, 4e-1
    p_in = dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od
    return wrapper((dest_wells[0][0], dest_wells[0][1], ahl_centers, p_in))

def residuals(p_in):
    y, t = sim_wrapper(p_in)
    sim_traces = get_traces(y)
    res = 0
    for lab in range(6):
        data_trace = clean_fit_vec[lab](t/60)
        res += np.sum(np.power(sim_traces[:,lab] - data_trace, 2))
    return res

p_0 = xa, xs, hR, kR, hC, kC, pa, leak, od0
opt_out = opt.minimize(residuals, p_0)
print(opt_out)


In [ ]:
opt_out.x - p_0